In [1]:
from pathlib import Path
import pickle
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
import pandas as pd
import numpy as np
from difflib import get_close_matches
from sklearn import preprocessing
from Levenshtein import distance

In [3]:
!wget https://www.kaggle.com/datasets/CooperUnion/anime-recommendations-database/download?datasetVersionNumber=1

"wget" �� ���� ����७��� ��� ���譥�
��������, �ᯮ��塞�� �ணࠬ��� ��� ������ 䠩���.


In [22]:
dataset_path = 'data/anime.csv'

In [23]:
data = Path('data')

In [24]:
df = pd.read_csv(dataset_path)
df = df.dropna()

In [25]:
df['rating'] = df['rating']*df['members']/(df['members']+10)


In [26]:
df.sort_values("members")

,anime_id,name,genre,type,episodes,rating,members
10444,34490,Sushi Azarashi,Comedy,TV,30,1.636364,12
10464,33662,Taka no Tsume 8: Yoshida-kun no X-Files,"Comedy, Parody",Movie,1,5.652174,13
8367,34259,Chotto Ugoku!? &quot;Futeneko&quot;,Comedy,OVA,1,3.777778,17
9805,34363,Ocha-ken: Ryokutto Monogatari,"Kids, Slice of Life",TV,13,5.333333,20
10500,33822,Ten ni Yaka Reru: E ni Yoru Shougen,"Drama, Historical",OVA,1,4.666667,20
...,...,...,...,...,...,...,...
159,6547,Angel Beats!,"Action, Comedy, Drama, School, Supernatural",TV,13,8.389883,717796
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.259883,793665
804,11757,Sword Art Online,"Action, Adventure, Fantasy, Game, Romance",TV,25,7.829912,893100
86,16498,Shingeki no Kyojin,"Action, Drama, Fantasy, Shounen, Super Power",TV,25,8.539905,896229


In [27]:
df_processed = pd.get_dummies(df, columns=['genre', 'type'])
df_processed = df_processed.drop(columns=['name', 'anime_id', 'episodes'])

x = df_processed.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x) / 2
df_processed = pd.DataFrame(x_scaled)

df_processed

,0,1,2,3,4,5,6,7,8,9,...,3227,3228,3229,3230,3231,3232,3233,3234,3235,3236
0,0.500000,0.098933,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0
1,0.493124,0.391384,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5
2,0.492453,0.056342,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5
3,0.487479,0.332161,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5
4,0.486822,0.074590,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12012,0.160911,0.000098,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0
12013,0.166933,0.000084,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0
12014,0.205101,0.000102,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0
12015,0.207855,0.000080,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0


In [28]:
knn = NearestNeighbors(p=1)
knn.fit(df_processed)

cos_sim = cosine_similarity(df_processed)
cos_sim

array([[1.        , 0.34569262, 0.33452861, ..., 0.15981177, 0.1617842 ,
        0.56439624],
       [0.34569262, 1.        , 0.62979443, ..., 0.14515441, 0.14693623,
        0.16260641],
       [0.33452861, 0.62979443, 1.        , ..., 0.15887457, 0.16083693,
        0.17800296],
       ...,
       [0.15981177, 0.14515441, 0.15887457, ..., 1.        , 0.99999356,
        0.54316659],
       [0.1617842 , 0.14693623, 0.16083693, ..., 0.99999356, 1.        ,
        0.54376391],
       [0.56439624, 0.16260641, 0.17800296, ..., 0.54316659, 0.54376391,
        1.        ]])

In [29]:
# dump
with open(data / 'knn.pkl', 'wb') as f:
    pickle.dump(knn, f)

# load
with open(data / 'knn.pkl', 'rb') as f:
    knn = pickle.load(f)

# dump
with open(data / 'cos_sim.pkl', 'wb') as f:
    pickle.dump(cos_sim, f)

# load
with open(data / 'cos_sim.pkl', 'rb') as f:
    cos_sim = pickle.load(f)

In [30]:
def description(array: np.ndarray) -> np.ndarray:
    print(f"shape: {array.shape}, mean: {array.mean()}, std: {array.std()}, min: {array.min()}, max: {array.max()}")
description(cos_sim)

shape: (12017, 12017), mean: 0.2533760433977871, std: 0.19072053270412442, min: 0.0, max: 1.0000000000000004


In [31]:
def normalize(matrix: np.ndarray) -> np.ndarray:
    new_matrix = matrix - matrix.min()

    new_matrix *= 1 / new_matrix.max()
    return new_matrix

cos_sim = normalize(cos_sim)
description(cos_sim)

shape: (12017, 12017), mean: 0.25337604339778697, std: 0.19072053270412437, min: 0.0, max: 1.0


  Obtaining dependency information for Levenshtein from https://files.pythonhosted.org/packages/2f/04/a557197d0ba601f09fdd9ef9174b61d87706a291cd1e1578728eda3f44ab/Levenshtein-0.23.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for rapidfuzz<4.0.0,>=3.1.0 from https://files.pythonhosted.org/packages/e5/7a/cf0db74bb2f3c1f78928fb5ed559e7c5cd24cb3143e649fa49bcb1a0276d/rapidfuzz-3.6.0-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/101.1 kB ? eta -:--:--
   ---- ----------------------------------- 10.2/101.1 kB ? eta -:--:--
   --------------------------- ----------- 71.7/101.1 kB 787.7 kB/s eta 0:00:01
   -------------------------------------- 101.1/101.1 kB 966.3 kB/s eta 0:00:00
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   --- ------------------------------------ 0.2/1.6 MB 3.1 MB/s eta 0:00:01
   ----------- ---------------------------- 0.5/1.6 MB 4.8 MB/s eta 0:00:01
   ------------------------------


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
_names = list(df['name'].unique())
def find_names(name: str, count: int) -> list[str]:
    names = get_close_matches(name, _names, n=count)
    df['distance'] = df['name'].apply(lambda x: distance(x, name))
    rows = df[df['name'].isin(names)].sort_values("distance", ascending=True)["name"].unique()
    return list(rows)[: count]


def find_name(name: str) -> str:
    found_names = find_names(name, count=1)
    if len(found_names) == 0:
        raise KeyError(f"Name not found: {name}")
    return found_names[0]

find_names("Naruto Shippuden", count=50), find_name("Ten Naruto: Death Note")

(['Naruto: Shippuuden',
  'Iruka to Shounen',
  'Naruto: Shippuuden Movie 1',
  'Naruto: Shippuuden Movie 2 - Kizuna',
  'Naruto Shippuuden: Sunny Side Battle'],
 'Fullmetal Alchemist')

In [ ]:
find_names("One Piece", count=50)

  Obtaining dependency information for rasa from https://files.pythonhosted.org/packages/b0/f6/1355c90be7d33738529920f2d848e446be19bf86eeff8327b3de43835384/rasa-3.6.15-py3-none-any.whl.metadata
  Using cached rasa-3.6.15-py3-none-any.whl.metadata (28 kB)
  Obtaining dependency information for CacheControl<0.13.0,>=0.12.9 from https://files.pythonhosted.org/packages/72/a2/28e0ef082f7d78253aded97933e1d7b94bab3c5be366e8afd6513de4028e/CacheControl-0.12.14-py2.py3-none-any.whl.metadata
  Using cached CacheControl-0.12.14-py2.py3-none-any.whl.metadata (2.2 kB)
  Obtaining dependency information for PyJWT[crypto]<3.0.0,>=2.0.0 from https://files.pythonhosted.org/packages/2b/4f/e04a8067c7c96c364cef7ef73906504e2f40d690811c021e1a1901473a19/PyJWT-2.8.0-py3-none-any.whl.metadata
  Using cached PyJWT-2.8.0-py3-none-any.whl.metadata (4.2 kB)
  Using cached SQLAlchemy-1.4.50-cp311-cp311-win_amd64.whl
  Using cached absl_py-1.4.0-py3-none-any.whl (126 kB)
  Using cached aio_pika-8.2.3-py3-none-any.whl

  error: subprocess-exited-with-error
  
  × Building wheel for matplotlib (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [564 lines of output]
      
      Edit mplsetup.cfg to change the build options; suppress output with --quiet.
      
      BUILDING MATPLOTLIB
            python: yes [3.11.7 (tags/v3.11.7:fa7a6f2, Dec  4 2023, 19:24:49) [MSC
                        v.1937 64 bit (AMD64)]]
          platform: yes [win32]
             tests: no  [skipping due to configuration]
            macosx: no  [Mac OS-X only]
      
      C:\Users\smrn2\AppData\Local\Temp\pip-build-env-8snhktpa\overlay\Lib\site-packages\setuptools\dist.py:674: SetuptoolsDeprecationWarning: The namespace_packages parameter is deprecated.
      !!
      
              ********************************************************************************
              Please replace its usage with implicit namespaces (PEP 420).
      
              See https://setuptools.pypa.io/en/latest/reference

In [52]:
def recomend_cos(target_name: str, top: int = 10):
    idx = df[df['name'] == target_name].index[0]
    score_series = pd.Series(cos_sim[idx]).sort_values(ascending = False)

    top = top + 1
    top_indexes = list(score_series.iloc[0:top].index)

    for i in top_indexes:
        yield df['name'].iloc[i], score_series[i]


def recomend_knn(target_name: str, top: int = 10):
    target_idx = df[df['name'] == target_name].index[0]
    row = df_processed.iloc[target_idx]

    distances, idxs = knn.kneighbors([row], top+1, return_distance=True)
    distances, idxs = distances[0], idxs[0]

    for i, idx in enumerate(idxs):
        yield df['name'].iloc[idx], distances[i]

In [53]:
def closest_to_best(n: int, best: int) -> None:
    best_sellers = df.sort_values(by='rating', ascending=False)[:best]
    for i in range(n):
        target_name = best_sellers['name'].iloc[np.random.randint(0, len(best_sellers))]
        print(target_name)
        for predict_name, conf in recomend_knn(target_name, top=4): # recomend_knn
            if predict_name == target_name:
                continue
            print(f'{round(conf, 5)}: {predict_name}')
        print()

closest_to_best(n=4, best=100)

Mob Psycho 100
1.00119: Usagi Drop
1.00175: Nana
1.00424: JoJo no Kimyou na Bouken (TV)
1.0054: Ghost in the Shell: Stand Alone Complex

Chihayafuru 2
0.04826: Chihayafuru
1.00325: Mononoke
1.00423: Slam Dunk
1.00839: One Outs

Gintama&#039;
0.02388: Gintama°
0.03777: Gintama&#039;: Enchousen
0.09879: Gintama
1.02385: Hajime no Ippo

Hunter x Hunter (2011)
0.1688: Hunter x Hunter
0.34668: Nano Invaders
1.01783: Shigatsu wa Kimi no Uso
1.01962: Clannad: After Story



In [55]:
def closest_to(name: str, n: int) -> None:
    target_name = find_name(name)
    print(target_name)

    for predict_name, conf in recomend_cos(target_name, top=n): # recomend_knn
        if predict_name == target_name:
            continue
        print(f'{round(conf, 5)}: {predict_name}')

closest_to(name="Kimi no Na wa.	", n=5)

Kimi no Na wa.
0.66373: Ookami Kodomo no Ame to Yuki
0.66364: Mononoke Hime
0.66347: Suzumiya Haruhi no Shoushitsu
0.6633: Koe no Katachi
0.66281: Sen to Chihiro no Kamikakushi


In [57]:
closest_to(name="Naruto", n=5)

Naruto
0.99666: Naruto: Shippuuden
0.69875: Death Note
0.696: Fullmetal Alchemist: Brotherhood
0.6948: Shingeki no Kyojin
0.68868: Code Geass: Hangyaku no Lelouch


In [ ]:
def best_from(count: int, field: str, name: str) -> list[str]:
    return list(df[df[field].str.contains(name)].sort_values(by='rating', ascending=False).name[:count])

best_from(count=10, field='genre', name='Romance')

['Kimi no Na wa.',
 'Clannad: After Story',
 'Shigatsu wa Kimi no Uso',
 'Rurouni Kenshin: Meiji Kenkaku Romantan - Tsuioku-hen',
 'Suzumiya Haruhi no Shoushitsu',
 'Monogatari Series: Second Season',
 'Howl no Ugoku Shiro',
 'Bakuman. 3rd Season',
 'Kara no Kyoukai 5: Mujun Rasen',
 'Yojouhan Shinwa Taikei']

In [ ]:
best_from(count=100, field='type', name='TV')

['Fullmetal Alchemist: Brotherhood',
 'Gintama°',
 'Steins;Gate',
 'Gintama&#039;',
 'Haikyuu!!: Karasuno Koukou VS Shiratorizawa Gakuen Koukou',
 'Hunter x Hunter (2011)',
 'Gintama&#039;: Enchousen',
 'Clannad: After Story',
 'Gintama',
 'Code Geass: Hangyaku no Lelouch R2',
 'Haikyuu!! Second Season',
 'Shigatsu wa Kimi no Uso',
 'Mushishi Zoku Shou 2nd Season',
 'Code Geass: Hangyaku no Lelouch',
 'Hajime no Ippo',
 'One Punch Man',
 'Cowboy Bebop',
 'Monogatari Series: Second Season',
 'Mushishi Zoku Shou',
 'Tengen Toppa Gurren Lagann',
 'Mushishi',
 'Great Teacher Onizuka',
 'Natsume Yuujinchou Go',
 'Natsume Yuujinchou Shi',
 'Hajime no Ippo: New Challenger',
 'Fate/Zero 2nd Season',
 'Monster',
 'Death Note',
 'Bakuman. 3rd Season',
 'Haikyuu!!',
 'Ansatsu Kyoushitsu (TV) 2nd Season',
 'Hajime no Ippo: Rising',
 'Natsume Yuujinchou San',
 'Ping Pong The Animation',
 'Boku dake ga Inai Machi',
 'Yojouhan Shinwa Taikei',
 'Re:Zero kara Hajimeru Isekai Seikatsu',
 'Rainbow: Nisha